In [3]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os

from datetime import date
print(date.today())

2025-12-08



```bash
```

In [5]:
sourcedir = '/mnt/disk7t/xwj/axolotl_rev/ly_111/'
datadir = '/mnt/disk7t/xwj/axolotl_rev/ly_111/'
os.chdir(datadir)

In [55]:
outlier = pd.read_csv(f'{datadir}/df_corrupt_ly111.txt', sep='\t', index_col=0)
outlier = outlier.dropna(subset=['row','col']).rename(columns={'col':'Sample','row':'Gene'})
outlier.head()
list_g = outlier['Gene'].unique().tolist()
list_s = outlier['Sample'].unique().tolist()
list_g.remove('VARS')

outlier = outlier.query('Gene in @cts.index')
outlier.to_csv(f'{datadir}/outlier_ly111_filtered_by_cts.txt', sep='\t')

In [59]:
cts = pd.read_csv(f'{datadir}/df_cts_ly_blood_g11525_s111.txt', sep='\t', index_col=0)
cts.index.name = 'Gene'
cts.shape, #outlier.shape
min_reads = 0
mask = (cts > min_reads).all(axis=1)
cts = cts[mask]
cts.shape, cts.index.name, cts.columns.name

((11310, 111), 'Gene', None)

In [ ]:
temp =pd.DataFrame( [ (g, s, cts.loc[g,s]) for g,s in zip(outlier['Gene'], outlier['Sample'])  if g in cts.index ] )
temp.sort_values(2)
cts[mask].loc[ list_g, list_s ].min(axis=1).describe()
cts.index.nunique()

/tmp/ipykernel_63766/2441311985.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  cts[mask].loc[ list_g, list_s ].min(axis=1).describe()


11310

In [60]:
use = 'ly111'
tissue_name_mapping = pd.DataFrame({
    'TISSUE_ID': [ 'B' ],
    'TISSUE_NAME': [ 'blood',]
})
tissues = pd.DataFrame(
    index = tissue_name_mapping.index,
    columns = ['TISSUE_ID', 'TISSUE_NAME', 'N_SAMPLE','N_GENE','CTS_FILE', 'OUTLIER_FILE',])

# 各组织cts表达矩阵和outlier
for idx, row in tissue_name_mapping.iterrows():
    tissue_id, tissue_name = row['TISSUE_ID'],  row['TISSUE_NAME']
    # (1) cts: expression matrix
    print( cts.shape, outlier.shape, outlier["Sample"].isin(cts.columns).sum(), outlier["Gene"].isin(cts.index).all(),)
    # (2) outlier: true outlier gene-sample pair in cts.    
    # sample meta table
    sample_list_interest = cts.columns.tolist()
    outlier_use = outlier.query('Sample in @sample_list_interest').copy()

    ctsfile = f'{datadir}/cts_{tissue_id}_s{cts.shape[1]}_g{cts.shape[0]}.tsv.gz'
    cts.to_csv(ctsfile, sep='\t')
    # outliers: part1 + part2
    outlierfile =f'{datadir}/outlier_{tissue_id}_sg{outlier_use.shape[0]}.tsv'
    outlier_use.to_csv(outlierfile, sep='\t')
    
    print(f'{tissue_name}/{tissue_id}/min_reads={min_reads} stable_ngene={cts.shape[0]} nsample={cts.shape[1]}')

    # print(outlierfile,ctsfile)
    tissues.loc[idx, :] = [ tissue_id, tissue_name, cts.shape[1], cts.shape[0],ctsfile, outlierfile]
    
tissues.to_csv(f'{datadir}/tissues_{use}.tsv', sep='\t')

(11310, 111) (52, 36) 52 True
blood/B/min_reads=0 stable_ngene=11310 nsample=111


## prepare folders and config file

In [57]:
# 准备结果目录和流程所需config文件
workdir = '/mnt/disk7t/xwj/axolotl_rev/'

# level 1
output_path = f'{workdir}/result/dataset_ly111'
# level 2
samples_path = f'{output_path}/samples'
task_config_path = f'{output_path}/task_config'
task_output_path = f'{output_path}/task_output'
metric_output_path = f'{output_path}/metric'

# print(workdir, samples_path, task_config_path, task_output_path, metric_output_path)
os.system(f'mkdir -p {samples_path} {task_config_path} {task_output_path} {metric_output_path}')
# os.system(f'chmod --silent -R 777 {task_output_path}')

0

In [61]:
# prepare task config file.
# task_config table have columns indicating output filenames of different methods
for i, row in tissues.iterrows():
    
    t  = row.TISSUE_ID
    tn = row.TISSUE_NAME
    ns = row.N_SAMPLE
    ng = row.N_GENE
    ctsfile = row.CTS_FILE
    outlierfile = row.OUTLIER_FILE

    prefix = f't{i:02d}_{t}_s{ns}_g{ng}' # id, number of samples, number of genes.
    
    cols = ['Dname','cts','samples','MyMethod','OUTRIDER','ABEILLE','OUTSINGLE']
    tasks = [0] # list of parallel tasks
    task_config = pd.DataFrame(index=tasks, columns=cols)
    task_config.index.name = 'task'

    task = 0 # 默认是全部样本, 所以only one task for this tissue  
    task_config.loc[task, 'Dname' ] = t
    task_config.loc[task, 'cts' ] = ctsfile
    task_config.loc[task, 'samples' ] = f'{samples_path}/{prefix}.txt'
    # create filenames
    task_config.loc[task, 'MyMethod'] = f'{task_output_path}/{prefix}/{task:03d}_mymethod.txt.gz'
    task_config.loc[task, 'OUTRIDER'] = f'{task_output_path}/{prefix}/{task:03d}_outrider.txt.gz'
    task_config.loc[task, 'OUTSINGLE'] = f'{task_output_path}/{prefix}/{task:03d}_outsingle.txt.gz'
    task_config.loc[task, 'ABEILLE'] = f'{task_output_path}/{prefix}/{task:03d}_abeille.txt.gz'

    # 0. create config & output folder of parallel tasks
    task_config.to_csv(f'{task_config_path}/{prefix}.config',sep='\t')
    os.system(f'mkdir -p {task_output_path}/{prefix}')
    
    # 1. sample ids of parallel tasks
    # 将task specific样本列表作为one row添加到DataFrame中. 默认task是全部样本
    cts = pd.read_csv(ctsfile, sep='\t',index_col=0)
    all_samples_df = pd.DataFrame(data=cts.columns.T.tolist()).transpose()
    all_samples_df.index = task_config.index
    all_samples_df.to_csv(f'{samples_path}/{prefix}.txt',sep='\t')
    

In [68]:
task_config
task_config_path, prefix

('/mnt/disk7t/xwj/axolotl_rev//result/dataset_ly111/task_config',
 't00_B_s111_g11310')

In [65]:
## 20231129 展示A group & B group samples
df_sample_orders = pd.read_csv('/mnt/disk7t/xwj/work/expression_outlier/output/data_liuyang/Number_col20231129.txt', 
                               sep='\t')
df_sample_orders

,Number,col
0,A01,AS35814
1,A02,AS23161440
2,A03,AS60159
3,A04,AS30833
4,A05,AS73212
...,...,...
106,B88,AS23341
107,B89,AS23340
108,B90,AS23158019
109,B91,AS43107


In [104]:
# toy dataset

# 准备结果目录和流程所需config文件
# level 1
output_path = '/mnt/disk7t/xwj/axolotl_rev/submit_code/testdata'
ctsfile = f'{output_path}/toy_small.tsv'
# level 2
samples_path = f'{output_path}/samples'
task_config_path = f'{output_path}/task_config'
task_output_path = f'{output_path}/task_output'
metric_output_path = f'{output_path}/metric'
os.system(f'mkdir -p {samples_path} {task_config_path} {task_output_path} {metric_output_path}')


# prepare task config file.
i = 0
t  = 'TOY'
tn = 'TOY'
ns = 128 
ng = 1000

cts = pd.read_csv(ctsfile, sep='\t',index_col=0)

prefix = f't{i:02d}_{t}_s{ns}_g{ng}' # id, number of samples, number of genes.

cols = ['Dname','cts','samples','MyMethod','OUTRIDER','ABEILLE','OUTSINGLE']
tasks = [0] # list of parallel tasks
task_config = pd.DataFrame(index=tasks, columns=cols)
task_config.index.name = 'task'

task = 0 # 默认是全部样本, 所以only one task for this tissue  
task_config.loc[task, 'Dname' ] = t
task_config.loc[task, 'cts' ] = ctsfile
task_config.loc[task, 'samples' ] = f'{samples_path}/{prefix}.txt'
# create filenames
task_config.loc[task, 'MyMethod'] = f'{task_output_path}/{prefix}/{task:03d}_mymethod.txt.gz'
task_config.loc[task, 'OUTRIDER'] = f'{task_output_path}/{prefix}/{task:03d}_outrider.txt.gz'
task_config.loc[task, 'OUTSINGLE'] = f'{task_output_path}/{prefix}/{task:03d}_outsingle.txt.gz'
task_config.loc[task, 'ABEILLE'] = f'{task_output_path}/{prefix}/{task:03d}_abeille.txt.gz'

# 0. create config & output folder of parallel tasks
task_config.to_csv(f'{task_config_path}/{prefix}.config',sep='\t')
os.system(f'mkdir -p {task_output_path}/{prefix}')

all_samples_df = pd.DataFrame(data=cts.columns.T.tolist()).transpose()
all_samples_df.index = task_config.index
all_samples_df.to_csv(f'{samples_path}/{prefix}.txt',sep='\t')


In [105]:
toy = pd.read_csv('/mnt/disk7t/xwj/axolotl_rev/ABEILLE/toy.csv',index_col=0)
toy.index.name = 'Gene'
toy = toy[(10 < toy ).all( axis =1)].head(1000)
toy.to_csv('/mnt/disk7t/xwj/axolotl_rev/submit_code/testdata/toy_small.tsv',sep='\t')
cts.shape

(1000, 128)